In [1]:
from dask.distributed import Client, SSHCluster
import sys
import dask
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import matplotlib.pyplot as plt

from dask import delayed
from time import time

## Functions

In [2]:
# ------ ALGORITHM FUNCTIONS
def load(path):
    return np.load(path, allow_pickle=True)["a"]

def R_yz(theta_rot, phi_rot):
    return np.array([[np.cos(phi_rot) * np.cos(theta_rot), -np.sin(phi_rot) * np.cos(theta_rot), np.sin(theta_rot)], 
                     [np.sin(phi_rot), np.cos(phi_rot), 0], 
                     [-np.sin(theta_rot) * np.cos(phi_rot), np.sin(theta_rot) * np.sin(phi_rot), np.cos(theta_rot)]])

def convert_pmt_ids(input_ids, conversion_ids):
    cd_ids  = np.array(conversion_ids["CdID"])
    pmt_ids = np.array(conversion_ids["PMTID"])
    mask    = np.isin(cd_ids, input_ids)
    return pmt_ids[mask]

def find_pmt_coord(pmt_positions, data_pmt_id):
    return pmt_positions[
        np.isin(pmt_positions.PMTID, data_pmt_id)
        ].loc[:, ["x", "y", "z"]].reset_index(drop=True).to_numpy()

# ------ DASK FUNCTIONS
def load_bag(path, Nevents):
    data_np = load(path)
    data_np = data_np[:, :Nevents]
    return [np.vstack([ data_np[j, i] for j in range(3)]) for i in range(data_np.shape[1])]


# ------ DISTRIBUTED ALGORITHM FUNCTIONS
def rotate_ev(data):

    nonzeros_inds = data[2] != 0.0
    data_pmt_id   = convert_pmt_ids(data[0][nonzeros_inds], conversion_ids)
    pmt_coord     = find_pmt_coord(pmt_positions, data_pmt_id)

    tot_charge = sum(data[1][nonzeros_inds])
    x_cc       = sum(pmt_coord[:,0] * data[1][nonzeros_inds]) / tot_charge
    y_cc       = sum(pmt_coord[:,1] * data[1][nonzeros_inds]) / tot_charge
    z_cc       = sum(pmt_coord[:,2] * data[1][nonzeros_inds]) / tot_charge

    theta_cc   = np.arctan2(
        np.sqrt((x_cc)**2+(y_cc)**2), z_cc
    )
    phi_cc     = np.arctan2(y_cc, x_cc) 

    theta_rot = -theta_cc + np.pi/2
    phi_rot   = -phi_cc
    
    # coord_new = np.matmul(R_yz(theta_rot, phi_rot), pmt_coord.T)
    coord_new = np.matmul(
        R_yz(theta_rot, phi_rot), pmt_coord.T
    )

    R = np.sqrt(np.sum(np.power(coord_new, 2), axis=0))

    charge_hitt = np.vstack([data[1], data[2]])
    charge_hitt = charge_hitt[:,nonzeros_inds]

    rotated = np.vstack([coord_new, R, charge_hitt])
    
    del coord_new
    del charge_hitt
    del pmt_coord
    
    return rotated



def mapping_single_event(rotated_ev):
    ####################
    # rotated_ev must be computed
    ####################
    
    N_max = 115
    
    coord_new   = rotated_ev[:3]
    charge_hitt = rotated_ev[4:, ].T
    R           = rotated_ev[3, ].mean()

    z_levels, step = np.linspace(coord_new[2,].min(), coord_new[2,].max(), 124, retstep=True)
    #z_levels       = z_levels.persist()
    image_mat      = np.zeros((230,124,2))

    #masks = 

    for j, z in enumerate(z_levels):
        mask = (np.abs(coord_new[2,] - z) < step)         #(np.abs(pmt_pos.z - z) < delta)
        if(not np.any(mask)): continue
        masked = coord_new[:,mask]


        Rz = (R**2 - z**2)
        Neff = 0 if Rz < 0 else N_max * np.sqrt(Rz) / R
        #ix = np.zeros(np.sum(mask), dtype=np.int32)
        ix = np.around( Neff * (np.arctan2(masked[1,], masked[0,]) / np.pi) + (N_max / 2) ) + 57
        ix = ix.astype(np.int32)
        #ix = ix.compute()
        if(np.any(ix >= 230)):
            ix[ix >= 230] = ix[ix >= 230] - 230

        image_mat[ix, j,] = charge_hitt[mask, ]

                # if np.isnan(mat[ix, i+1]):
                #     mat[ix, i+1] = row.id
                # else:
                #     mat[ix, 123 if i else i] = row.id

    del rotated_ev
    return image_mat

## Load mapping data

In [3]:
data_folder       = "/root/data/data/real/train/data/"
pmt_pos_fname     = "/root/distributed-juno/cluster/PMTPos_CD_LPMT.csv"
pmt_id_conv_fname = "/root/distributed-juno/cluster/PMT_ID_conversion.csv"
train_data_fname  = "raw_data_train_4.npz"

In [4]:
pmt_positions     = pd.read_csv(pmt_pos_fname)
pmt_id_conversion = pd.read_csv(pmt_id_conv_fname)
conversion_ids    = pd.read_csv(pmt_id_conv_fname)

## Ideas

Varying the following parameters
*   `n_workers`
*   `n_threads`
*   `n_partitions`

Then
1. study total time 
2. study each function separately through a mid-point `compute()`
3. ...?

## Grid search

In [5]:
w_max      = 8
workers    = np.arange(1, w_max+1)
threads    = [w_max - worker + 1 for worker in workers]

Nevents    = 1000

ips        = ["10.67.22.39", "10.67.22.74", "10.67.22.27", "10.67.22.91", "10.67.22.60"]
ssh_opt    = {"known_hosts": "/root/.ssh/known_hosts"}
sched_port = {"dashboard_address": ":8787"}
# worker_par = {"nthreads": 1, "n_workers": 4}

In [6]:
worker_list        = []
thread_list        = []
load_time_list     = []
rotation_time_list = []
mapping_time_list  = []
compute_time_list  = []
total_time_list    = []

for n_w in workers:
    for n_t in threads:
    
        print(f"Workers: {n_w}\tThreads: {n_t}")

        worker_par = {"nthreads": int(n_t), "n_workers": int(n_w)}

        cluster = SSHCluster(
            ips,
            connect_options   = ssh_opt,
            worker_options    = worker_par,
            scheduler_options = sched_port
        )
        client = Client(cluster)
        print(client)

        print("\n\nPROCESSING:\n")
        #################################
        time_0  = time()

        start   = time()
        data_db = db.from_sequence(load_bag(data_folder + train_data_fname, Nevents=Nevents), npartitions=16)
        stop    = time()

        load_time = stop-start
        print("Load time:\t", load_time)

        start   = time()
        rotated = db.map(rotate_ev, data_db)
        stop    = time()

        rotation_time = stop-start
        print("Rotation time:\t", rotation_time)

        start   = time()
        mapped  = db.map(mapping_single_event, rotated)
        stop    = time()

        projection_time = stop-start
        print("Mapping time:\t", projection_time)

        start   = time()
        images  = mapped.compute()
        stop    = time()

        compute_time = stop-start
        print("Compute time:\t", compute_time)



        time_1 = time()
        total_time = time_1 - time_0
        print("\n\nTotal time:\t", total_time)
        #################################

        client.shutdown()
        
        worker_list.append(n_w)        
        thread_list.append(n_t)        
        load_time_list.append(load_time)     
        rotation_time_list.append(rotation_time) 
        mapping_time_list.append(projection_time)  
        compute_time_list.append(compute_time)  
        total_time_list.append(total_time)   

Workers: 1	Threads: 8


2022-07-06 18:04:40,255 - distributed.deploy.ssh - INFO - 2022-07-06 18:04:40,253 - distributed.http.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
2022-07-06 18:04:40,314 - distributed.deploy.ssh - INFO - 2022-07-06 18:04:40,313 - distributed.scheduler - INFO - State start
2022-07-06 18:04:40,320 - distributed.deploy.ssh - INFO - 2022-07-06 18:04:40,319 - distributed.scheduler - INFO - Clear task state
2022-07-06 18:04:40,322 - distributed.deploy.ssh - INFO - 2022-07-06 18:04:40,321 - distributed.scheduler - INFO -   Scheduler at:   tcp://10.67.22.39:37329
2022-07-06 18:04:43,475 - distributed.deploy.ssh - INFO - 2022-07-06 18:04:43,473 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.74:42371'
2022-07-06 18:04:43,533 - distributed.deploy.ssh - INFO - 2022-07-06 18:04:43,531 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.91:34105'
2022-07-06 18:04:43,797

<Client: 'tcp://10.67.22.39:37329' processes=2 threads=16, memory=15.54 GiB>


PROCESSING:

Load time:	 12.617675304412842
Rotation time:	 0.026809215545654297
Mapping time:	 0.0014886856079101562
Compute time:	 14.596028804779053


Total time:	 27.24235701560974
Workers: 1	Threads: 7


2022-07-06 18:05:17,497 - distributed.deploy.ssh - INFO - 2022-07-06 18:05:17,496 - distributed.http.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
2022-07-06 18:05:17,554 - distributed.deploy.ssh - INFO - 2022-07-06 18:05:17,552 - distributed.scheduler - INFO - State start
2022-07-06 18:05:17,558 - distributed.deploy.ssh - INFO - 2022-07-06 18:05:17,557 - distributed.scheduler - INFO - Clear task state
2022-07-06 18:05:17,561 - distributed.deploy.ssh - INFO - 2022-07-06 18:05:17,557 - distributed.scheduler - INFO -   Scheduler at:   tcp://10.67.22.39:37497
2022-07-06 18:05:18,677 - distributed.deploy.ssh - INFO - 2022-07-06 18:05:18,675 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.91:37533'
2022-07-06 18:05:18,744 - distributed.deploy.ssh - INFO - 2022-07-06 18:05:18,743 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.74:45465'
2022-07-06 18:05:19,191

<Client: 'tcp://10.67.22.39:37497' processes=2 threads=14, memory=15.54 GiB>


PROCESSING:

Load time:	 10.980205059051514
Rotation time:	 0.020067930221557617
Mapping time:	 0.001323699951171875


...

Load time:	 8.167396783828735
Rotation time:	 0.01988697052001953
Mapping time:	 0.0008821487426757812
Compute time:	 5.826564073562622


Total time:	 14.015439987182617
Workers: 8	Threads: 1


2022-07-06 18:26:20,245 - distributed.deploy.ssh - INFO - 2022-07-06 18:26:20,244 - distributed.http.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
2022-07-06 18:26:20,302 - distributed.deploy.ssh - INFO - 2022-07-06 18:26:20,301 - distributed.scheduler - INFO - State start
2022-07-06 18:26:20,307 - distributed.deploy.ssh - INFO - 2022-07-06 18:26:20,306 - distributed.scheduler - INFO - Clear task state
2022-07-06 18:26:20,308 - distributed.deploy.ssh - INFO - 2022-07-06 18:26:20,307 - distributed.scheduler - INFO -   Scheduler at:   tcp://10.67.22.39:39223
2022-07-06 18:26:21,418 - distributed.deploy.ssh - INFO - 2022-07-06 18:26:21,414 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.91:34223'
2022-07-06 18:26:21,425 - distributed.deploy.ssh - INFO - 2022-07-06 18:26:21,422 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.91:43655'
2022-07-06 18:26:21,428

<Client: 'tcp://10.67.22.39:39223' processes=17 threads=17, memory=33.03 GiB>


PROCESSING:



2022-07-06 18:26:24,601 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


Load time:	 8.277276515960693
Rotation time:	 0.020277738571166992
Mapping time:	 0.002072572708129883
Compute time:	 3.812199831008911


Total time:	 12.11221718788147


In [7]:
grid_results = pd.DataFrame(
    {
        "n_workers"     : worker_list,
        "n_threads"     : thread_list,
        "load_time"     : load_time_list,
        "rotation_time" : rotation_time_list,
        "mapping_time"  : mapping_time_list,
        "compute_time"  : compute_time_list,
        "total_time"    : total_time_list,
    }
)
grid_results

,n_workers,n_threads,load_time,rotation_time,mapping_time,compute_time,total_time
0,1,8,12.617675,0.026809,0.001489,14.596029,27.242357
1,1,7,10.980205,0.020068,0.001324,14.149490,25.152314
2,1,6,11.064290,0.020224,0.001274,13.285676,24.372086
3,1,5,9.479378,0.019058,0.001214,13.029329,22.530997
4,1,4,9.319416,0.019973,0.005122,12.924639,22.269490
...,...,...,...,...,...,...,...
59,8,5,8.057083,0.021041,0.000930,6.195173,14.274738
60,8,4,8.163486,0.018986,0.000982,6.135245,14.319311
61,8,3,8.386006,0.020056,0.000630,5.951200,14.358195
62,8,2,8.167397,0.019887,0.000882,5.826564,14.015440


In [8]:
grid_results.to_csv("grid_results.csv", index=False)

## Shutdown cluster

In [9]:
# client.shutdown()

In [10]:
# client.restart()

2022-07-06 18:26:46,443 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
2022-07-06 18:27:06,524 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
